# Sandbox
For playing around during development.

In [1]:
%pylab inline
import pysd
print pysd.__version__
print pysd.__file__
import pandas as pd
import os
os.path.abspath(pysd.__file__)

Populating the interactive namespace from numpy and matplotlib
0.4.2
pysd\__init__.pyc


'C:\\Users\\BOSMY\\Documents\\pysd-master\\New PySD\\2-3-16 pysd-subscript-parsimonious\\pysd\\__init__.pyc'

In [2]:
#mdl_file = 'tests/old_tests/vensim/Teacup.mdl'
# mdl_file = 'tests/test-models/tests/stamps/fasttest.mdl'
mdl_file = 'tests/stamps/pdatvensim8k_RW_v2_Unchanged.mdl'
# mdl_file = 'tests/stamps/basenov9.mdl'
#mdl_file = 'tests/test-models/tests/subscript_2d_arrays/test_subscript_2d_arrays.mdl'
#mdl_file = 'tests/test-models/tests/subscript_individually_defined_1_of_2d_arrays/subscript_individually_defined_1_of_2d_arrays.mdl'
#mdl_file = 'tests/test-models/tests/subscript_individually_defined_1d_arrays/subscript_individually_defined_1d_arrays.mdl'
model = pysd.read_vensim(mdl_file)

In [3]:
stocks=model.run(flatten_subscripts=True)

In [8]:
stocks.to_excel("Fruits Sales.xlsx")

In [1]:
string='abcadwegwe'
string.split(',')

['abcadwegwe']

In [7]:
model.components.onboard_equiv_crew_working_maintenance()

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [8]:
np.shape(pysd.functions.shorthander(model.components.effect_on_productivity_of_rate(),
                                    model.components.effect_on_productivity_of_rate.dimension_dir,
                                    model.components.crew_productivity_by_type.dimension_dir,
                                    model.components._subscript_dict)[:,0])

(9L,)

In [9]:
np.shape(pysd.functions.shorthander(model.components.impact_of_skill_on_pdy(),
                                   model.components.impact_of_skill_on_pdy.dimension_dir,
                                   model.components.crew_productivity_by_type.dimension_dir,
                                   model.components._subscript_dict))

(9L, 2L)

In [1]:
from pysd import functions

In [21]:
_t = 4

In [14]:
def time():
    return _t

In [15]:
functions.__builtins__.update({'time':time})

In [22]:
functions.step(10, 4)

10

In [23]:

from pysd import functions

def time():
    return _t

_t = 2

functions.__builtins__.update({'time':time})

print functions.step(10, 4)

_t = 5

print functions.step(10, 4)

0
10
